In [1]:
import os

with open("./tavily_key.txt", "r") as f:
    tavily_key = f.readline()
os.environ["TAVILY_API_KEY"] = tavily_key

In [2]:
from tavily import TavilyClient

client = TavilyClient(api_key=tavily_key)

In [4]:
result = client.search("What is in Nvidia's new Blackwell GPU?", include_answer=True)
result['answer']

"Nvidia's new Blackwell GPU, specifically the Blackwell B200 tensor core chip, is designed to be the company's most powerful single-chip GPU with 208 billion transistors. It is reported to reduce AI inference operating costs and energy consumption by up to 25 times compared to the previous H100 GPU. The Blackwell GPUs promise significant performance improvements, ranging from 7 to 30 times better than the H100 GPUs, depending on the task. The Blackwell GPUs will be part of a server product called the GB200 NVL72, which will feature 72 Blackwell GPUs connected via Nvidia's upgraded NVLink technology to deliver high bidirectional throughput of 1.8TB/s. The Blackwell GPU is positioned to supercharge AI training and is expected to be adopted by major organizations such as Amazon Web Services, Google, Meta, Microsoft, and others."

Regular search vs Agentic Search

In [5]:
city = 'Bengaluru'

query = f"""what is the current weather in {city}
Should I travel there today?
"""

In [6]:
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

ddg = DDGS()

def search(query, max_results=3):
    results = ddg.text(query, max_results=max_results)
    return [i['href'] for i in results]

for i in search(query):
    print(i)

https://www.accuweather.com/en/in/bengaluru/204108/weather-forecast/204108
https://www.easeweather.com/asia/india/karnataka/bangalore-urban/bengaluru/today
https://www.accuweather.com/en/in/bengaluru/204108/weather-today/204108


In [ ]:
def scrape_weather_info(url):
    """Scrape content from the given URL"""
    if not url:
        return "Weather information could not be found."
    
    # fetch data
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    # parse result
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup



In [12]:
url = search(query=query)[0]

In [13]:
soup = scrape_weather_info(url)

In [14]:
print(f"Website: {url}\n\n")

Website: https://www.accuweather.com/en/in/bengaluru/204108/weather-forecast/204108




In [15]:
print(soup)


<!DOCTYPE html>

<html class="accuweather" lang="en-us">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8">
<link href="https://www.accuweather.com/en/in/bengaluru/204108/weather-forecast/204108" rel="canonical"/>
<title>Bengaluru, Karnataka, India Weather Forecast | AccuWeather</title>
<meta content="Bengaluru, Karnataka, India Weather Forecast, with current conditions, wind, air quality, and what to expect for the next 3 days." name="Description"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport">
<meta content="origin" name="referrer"/>
<meta content="AccuWeather" property="fb:profile_id"/>
<meta content="132437483467956" property="fb:app_id"/>
<meta content="71781612888" property="fb:pages"/>
<meta content="website" property="og:type"/>
<meta content="Bengaluru, Karnataka, India Weather Forecast | AccuWeather" property="og:title"/>
<meta content="Bengaluru, Karnataka, India Weather Forecast, with current conditions, w

In [16]:
weather_data = []
for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

# combine all elements into a single string
weather_data = "\n".join(weather_data)

# remove all spaces from the combined text
weather_data = re.sub(r'\s+', ' ', weather_data)
    
print(f"Website: {url}\n\n")
print(weather_data)

Website: https://www.accuweather.com/en/in/bengaluru/204108/weather-forecast/204108


Bengaluru, Karnataka Bengaluru Karnataka Around the Globe Around the Globe Hurricane Tracker Severe Weather Radar & Maps News & Features Astronomy Business Climate Health Recreation Sports Travel Video Today Tonight's Weather Sun, Nov 3 Increasing cloudiness Lo: 19° Tomorrow: Hazy sunshine Hi: 29° Current Weather 9:44 PM Looking Ahead Showers and thunderstorms in the area Wednesday evening Bengaluru Weather Radar & Maps Hourly Weather 10-Day Weather Forecast Tonight 11/3 Increasing cloudiness Mon 11/4 Hazy sunshine Night: Clear Tue 11/5 Hazy sunshine Mostly clear Wed 11/6 Hazy sunshine A couple of t-showers early Thu 11/7 Partly sunny and nice Clear to partly cloudy Fri 11/8 Partly sunny and pleasant Partly cloudy Sat 11/9 Cloudy with a stray t-storm Mainly clear Sun 11/10 Times of clouds and sun Partly cloudy Mon 11/11 Mostly sunny Partly cloudy Tue 11/12 A t-storm around in the p.m. Mostly cloudy Su

Agentic search tool

In [17]:
result = client.search(query=query, max_results=1)

data = result['results'][0]['content']
data

'Today. Bengaluru will have a warm day today with temperatures ascending to 28 C and descending to around 21 after sunset. Gentle breezes are expected across Bengaluru today, with speeds reaching around . Rain is highly likely at 88%, but with light accumulations around 1.66 mm.'

What an agent would like to see is more of a json formatted output though